In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/nudt/.julia/packages/Plots/kyYZF/src/backends.jl:372


In [2]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)

In [3]:
using Base.Threads;nthreads()

1

In [4]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
dot_num=30
initial_state=kron([1.0 0 0;0 0 0;0 0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3*dim,:)
end


n=kron(eye(3),a₊(dim)*a₋(dim))
en=kron([0 0 0;0 0 0;0 0 1],eye(dim))
rr=kron([0 0 0;0 1 0;0 0 0],eye(dim))
η1=-0.15
η2=0.15
a=a₊(dim)+a₋(dim)
ag2=kron([0 0 0;0 0 0;1 0 0],exp(im*η1*a))
ag3=kron([0 0 1;0 0 0;0 0 0],exp(-im*η1*a))
ag4=ag2+ag3
ar2=kron([0 0 0;0 0 0;0 1 0],exp(im*η2*a))
ar3=kron([0 0 0;0 0 1;0 0 0],exp(-im*η2*a))
ar4=ar2+ar3

E=eye(3*dim)
γ1=20/3*ν
γ2=40/3*ν

function diss1()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η1*theta*a)
        aup=kron([0 0 0;0 0 0;1 0 0],jump')
        adown=kron([0 0 1;0 0 0;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η2*theta*a)
        aup=kron([0 0 0;0 0 0;0 1 0],jump')
        adown=kron([0 0 0;0 0 1;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

dissipation1=γ1*diss1()
dissipation2=γ2*diss2()

# Δg=(Ωr^2+Ωg^2)/ν-ν
# Δr=Δg

function eit_Lindbladian_vec(Ωg,Ωr,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

instate=dm2vec(initial_state);


In [5]:
function sweep_detuning_50()
    ttot=50
    initial_x=[5.0 16]
    sweep_num=120
    start=30
    stop=150
    d=[]
    somega_g=[]
    somega_r=[]
    sn=[]
    for i in 1:sweep_num
        println(" ")
        println(i," ")
        m=start+(i-1)*(stop-start)/sweep_num
        function feval(x)
            f=exp(ttot*eit_Lindbladian_vec(x[1],x[2],m))*instate
            nbose=real(tr(n*vec2dm(f)))
            print(" f ",nbose," ")
            return nbose
        end        

        a=optimize(feval,initial_x,LBFGS())
        initial_x=Optim.minimizer(a) 
        append!(somega_g,initial_x[1])  
        append!(somega_r,initial_x[2])
        append!(sn,Optim.minimum(a)[1])
        append!(d,m)
        df1=DataFrame(delta=d,omega_g=somega_g,omega_r=somega_r,n=sn);CSV.write("eit_sweep_detuning_50.csv", df1);
    end
    return somega_g,somega_r,sn,d
end
@time res=sweep_detuning_50()


 
1 
 f 0.7782801747795477  f 0.7782811477940087  f 0.7782877687412247  f 0.7782735537690395  f 0.7782806612757768  f 0.7726220748501453  f 0.7726230675193771  f 0.7726296877768711  f 0.7726154545381915  f 0.7726225711736515  f 0.7496964946675693  f 0.7496975742410394  f 0.7497041246187639  f 0.7496899442640905  f 0.749697034441675  f 0.6306672713365051  f 0.6306686132758584  f 0.6306749467623823  f 0.6306609377208706  f 0.6306679422897334  f 0.36766533978084875  f 0.36767697464836546  f 0.36766631631315677  f 0.3676759981324309  f 0.3676711571865142  f 0.2801608460840538  f 0.2801603146522974  f 0.2801642718729122  f 0.28015688906551967  f 0.28016058034983005  f 0.9610801389397975  f 0.9610807819054149  f 0.9610792136312372  f 0.9610817072030579  f 0.961080460423325  f 0.5692963620431782  f 0.5693024361312167  f 0.5693055707797343  f 0.5692932276982726  f 0.5692993990801319  f 1.0409351942170724  f 1.0409350282314442  f 1.0409351097055062  f 1.0409351127458322  f 1.0409351112257024  f

 f 0.18148950423861732  f 0.18149137464140455  f 0.1814900778748927  f 0.18149080077938767  f 0.18149043929290162  f 0.18118522096340087  f 0.18118689362983667  f 0.18118573885992245  f 0.18118637550639996  f 0.18118605714914937  f 0.1803219785952343  f 0.18032286032225447  f 0.18032227051790758  f 0.18032256816814818  f 0.18032241930982462  f 0.1841401028518743  f 0.18413704993478697  f 0.18413920256479444  f 0.18413794997298266  f 0.1841385762391794  f 0.18001616534158543  f 0.1800161755443346  f 0.18001620276966823  f 0.18001613788014562  f 0.1800161702925792  f 0.1800109683666574  f 0.18001100677220616  f 0.18001100952689078  f 0.18001096537523348  f 0.18001098741869176  f 0.18000707955733672  f 0.1800072314378336  f 0.18000713599045468  f 0.1800071747652058  f 0.18000715534549633  f 0.18000535684607727  f 0.1800054678347025  f 0.18000540772840884  f 0.18000541671378267  f 0.18000541218875268  f 0.17999899636614  f 0.1799990501156854  f 0.17999902091054512  f 0.17999902533072082  f

 f 0.18330909034861814  f 0.18331102864896498  f 0.18330971138738317  f 0.18331040733532217  f 0.18331005932748148  f 0.18302682928851247  f 0.1830285605749544  f 0.18302738832874646  f 0.18302800125842444  f 0.18302769475997432  f 0.18222787377012387  f 0.18222877669628312  f 0.1822281816920083  f 0.18222846849307536  f 0.1822283250597534  f 0.18584374041668256  f 0.18584051924508022  f 0.1858427295418433  f 0.18584152981944965  f 0.1858421296516232  f 0.18194980269768868  f 0.18194980718360718  f 0.1819498328620775  f 0.1819497767326073  f 0.18194980476547262  f 0.18194616572471634  f 0.18194619907232606  f 0.1819462013929651  f 0.18194616311695963  f 0.18194618222305964  f 0.18194415109924883  f 0.18194430052458105  f 0.18194420909248493  f 0.18194424224164027  f 0.18194422563519497  f 0.1819425455712487  f 0.18194264806688526  f 0.1819425945023479  f 0.18194259884720693  f 0.18194259664285253  f 0.18193766009813675  f 0.18193771711538723  f 0.18193768723251796  f 0.1819376896904559

 f 0.1892180788351922  f 0.1892200794008578  f 0.18921874249519094  f 0.18921941541427487  f 0.18921907892121437  f 0.1889542113467821  f 0.18895599572923039  f 0.18895480710635706  f 0.18895539964150107  f 0.18895510334066995  f 0.1882097298679417  f 0.18821064895679324  f 0.1882100508339192  f 0.18821032765700627  f 0.18821018921301516  f 0.19167575993035077  f 0.1916723702952373  f 0.19167464427105563  f 0.19167348560069672  f 0.1916740649074013  f 0.18795676010856374  f 0.1879567592434884  f 0.1879567837186212  f 0.18795673529404117  f 0.1879567594748256  f 0.18795422700814196  f 0.1879542556345643  f 0.1879542575777454  f 0.1879542247250612  f 0.18795424111987186  f 0.18795370079554907  f 0.18795384796085388  f 0.1879537594720887  f 0.18795378894223905  f 0.1879537741756264  f 0.18795203036918084  f 0.18795212167707065  f 0.18795207577475076  f 0.18795207593041  f 0.18795207582107779  f 0.1879484822749372  f 0.1879485399266447  f 0.18794851086438955  f 0.18794851099439308  f 0.187

 f 0.2006258830987164  f 0.20062794972674117  f 0.2006265923418863  f 0.20062724009039049  f 0.20062691618302372  f 0.20038217922883886  f 0.20038401958319627  f 0.20038281406214392  f 0.20038338435505834  f 0.2003830991756937  f 0.19969719968038083  f 0.19969813413885795  f 0.19969753366769283  f 0.1996977997504471  f 0.1996976666770699  f 0.20298983804357276  f 0.20298626265202233  f 0.20298860756726725  f 0.20298749270690658  f 0.20298805010918675  f 0.19947089136101465  f 0.19947088472556695  f 0.19947090790914979  f 0.1994708677704976  f 0.19947088780872205  f 0.19946927739964615  f 0.1994693007399708  f 0.19946930227095114  f 0.19946927546135515  f 0.1994692888350776  f 0.1994700057495319  f 0.19947014947382116  f 0.1994700641280239  f 0.19947009068613986  f 0.1994700773759302  f 0.19946817983485607  f 0.1994682549644548  f 0.1994682191050662  f 0.19946821528604283  f 0.19946821716449697  f 0.1994659848364654  f 0.19946603872023536  f 0.19946601292753724  f 0.19946601021958604  f

...

 f 0.505765832779785  f 0.5057670047951048  f 0.5057662042244877  f 0.5057666327486371  f 0.5057664184338938  f 0.5057347150959527  f 0.5057358124861802  f 0.5057350626407945  f 0.5057354643386958  f 0.5057352634368627  f 0.5056307309814645  f 0.5056315288671659  f 0.5056309826022524  f 0.5056312766405215  f 0.5056311295681538  f 0.5056056595951559  f 0.5056049372593078  f 0.5056054240134709  f 0.5056051722187553  f 0.5056052980612941  f 0.5055143612552783  f 0.5055143577772103  f 0.5055143561478869  f 0.5055143622699125  f 0.5055143591549464  f 0.5055143313175334  f 0.505514325802178  f 0.5055143258592086  f 0.5055143306458287  f 0.5055143281983343  f 0.5055142599366029  f 0.5055142462709289  f 0.5055142530760072  f 0.5055142525172447  f 0.5055142527425134  f 0.5055150650633803  f 0.5055150106140845  f 0.5055150512115854  f 0.5055150238525878  f 0.5055150374776416  f 0.5055142531460355  f 0.5055142356150711  f 0.5055142456207774  f 0.5055142425258119  f 0.5055142440193238  f 0.5055141

 f 0.5204727566506203  f 0.5204738827547365  f 0.5204731074900043  f 0.5204735313248874  f 0.5204733193518615  f 0.5204445830746514  f 0.5204456396812404  f 0.5204449120481988  f 0.520445310116184  f 0.5204451110266364  f 0.5203498580557273  f 0.520350635702127  f 0.52035009927059  f 0.5203503938924439  f 0.5203502465255446  f 0.5203106882709981  f 0.5203100489997727  f 0.5203104837216083  f 0.5203102529386882  f 0.5203103682723302  f 0.5202383277987317  f 0.5202383254858362  f 0.5202383236708266  f 0.5202383290101251  f 0.520238326283362  f 0.5202383072337647  f 0.5202383026021111  f 0.5202383026428159  f 0.5202383065895059  f 0.5202383045590534  f 0.5202382706772636  f 0.5202382567702302  f 0.5202382642357554  f 0.5202382626083166  f 0.5202382633649401  f 0.5202391855774205  f 0.5202391252668324  f 0.5202391698999123  f 0.520239140341473  f 0.5202391550633463  f 0.5202382708240116  f 0.5202382563306948  f 0.5202382642655452  f 0.5202382622857966  f 0.5202382632185687  f 0.52023822177

 f 0.5346369989320428  f 0.5346380834204802  f 0.5346373312744551  f 0.5346377504978436  f 0.5346375408274884  f 0.5346113618284382  f 0.5346123813160113  f 0.5346116740737885  f 0.5346120684897179  f 0.5346118712231978  f 0.534524689422815  f 0.5345254479706287  f 0.5345249210080582  f 0.5345252158010388  f 0.5345250683456395  f 0.534475485011249  f 0.5344749173713395  f 0.5344753068437629  f 0.5344750949383775  f 0.5344752008305493  f 0.5344177870904184  f 0.5344177858767808  f 0.5344177839244981  f 0.5344177884491985  f 0.5344177861268375  f 0.534417774090593  f 0.5344177703658515  f 0.5344177703731884  f 0.534417773489801  f 0.5344177718714421  f 0.5344177642999218  f 0.534417750529875  f 0.5344177583774157  f 0.5344177558589577  f 0.5344177570581201  f 0.5344177602300392  f 0.5344177492796978  f 0.5344177549259543  f 0.5344177539897778  f 0.5344177543978825  f 0.53441773140112  f 0.5344177245405329  f 0.5344177279676187  f 0.534417727380462  f 0.5344177276140099  f 0.5344177390848

 f 0.5509439482832102  f 0.5509449876458927  f 0.5509442607955696  f 0.5509446745640378  f 0.5509444676181425  f 0.5509209200983983  f 0.5509218991765386  f 0.5509212143500016  f 0.550921604354784  f 0.5509214092906276  f 0.5508426058892115  f 0.5508433429332862  f 0.5508428268499123  f 0.5508431213990305  f 0.550842974062261  f 0.5507852015042518  f 0.5507847076664771  f 0.5507850500885624  f 0.550784858492889  f 0.5507849542265527  f 0.5507409592719084  f 0.5507409592964783  f 0.5507409572245033  f 0.5507409607609409  f 0.5507409589292362  f 0.5507409529091507  f 0.5507409502759328  f 0.5507409502245666  f 0.550740952377774  f 0.5507409512377281  f 0.5507409651414247  f 0.5507409518760449  f 0.5507409599082245  f 0.5507409565264659  f 0.5507409581537647  f 0.5507409504818457  f 0.5507409437524889  f 0.5507409468153692  f 0.5507409468361816  f 0.5507409467623923  f 0.5507409391688238  f 0.550740934156972  f 0.5507409363635529  f 0.5507409363793613  f 0.5507409363081783  f 0.5507409244

(Any[9.684964827684135, 9.858127360050988, 10.029148855529353, 10.198036977386675, 10.364810266295994, 10.52949537737122, 10.692124914461635, 10.852735639063416, 11.011367177280027, 11.168060857817077  …  21.02172272785475, 21.089373266763634, 21.156819552238545, 21.22406455595881, 21.29111002343623, 21.35795792166777, 21.424610435375207, 21.491069564851294, 21.557337064103223, 21.623414870055615], Any[6.383200117150157, 6.427379168907065, 6.470795479231739, 6.513528171014302, 6.555645027301617, 6.5972048367391585, 6.638259278656971, 6.678854463397311, 6.719032098079363, 6.758830491571497  …  12.002973551037561, 12.055093327752264, 12.106902888140578, 12.15840452054854, 12.209603789380967, 12.26050464209926, 12.311110039261182, 12.361425072521223, 12.411453878121481, 12.461199271347448], Any[0.18226438284540655, 0.18108139675463322, 0.18033977772619597, 0.1799971012835167, 0.18001620998717008, 0.18036439739240218, 0.18101273628040823, 0.18193552349004488, 0.18310981927779946, 0.1845150